# Übung 1: Berechnung einer absoluten Positionierung mit Code-Messungen

In [2]:
# Requirements 
# pip install git+https://github.com/GNSSpy-Project/gnsspy
# pip install pyunpack
# pip install georinex

import gnsspy as gp
# import georinex
import geopandas
import matplotlib.pyplot as plt
import math

In [7]:
# Dataimport
station = gp.read_obsFile("ONSA0320.11O")
ephemerides = gp.read_sp3File("G3_11032.PRE")

ONSA0320.11O exist in working directory | Reading...
Line 19 :                             4  4
Unexpected format between epochs! Line 20 is deleted!
Line 24 :                             4  4
Unexpected format between epochs! Line 25 is deleted!
Line 29 :                             4  4
Unexpected format between epochs! Line 30 is deleted!
Line 34 :                             4  4
Unexpected format between epochs! Line 35 is deleted!
Line 39 :                             4  4
Unexpected format between epochs! Line 40 is deleted!
Line 44 :                             4  4
Unexpected format between epochs! Line 45 is deleted!
Line 49 :                             4  4
Unexpected format between epochs! Line 50 is deleted!
Line 54 :                             4  4
Unexpected format between epochs! Line 55 is deleted!
Line 59 :                             4  4
Unexpected format between epochs! Line 60 is deleted!
Line 64 :                             4  4
Unexpected format between epoch

IndexError: list index out of range

In [18]:
def calculateLatLong(earth_fixed_coords):
    lat = list()
    long = list()

    for x, y, z in earth_fixed_coords:
        lat.append(math.atan2(z, math.sqrt(x**2 + y**2)))
        long.append(math.atan2(y, x))

    return lat, long

def plotGroundTrack(name, desc, lat, long, min, max):
    countries = geopandas.read_file(geopandas.datasets.get_path("naturalearth_lowres"))

    countries.plot(color = "grey")
    plt.scatter(long, lat, color = "purple")

    plt.grid()
    plt.ylim(-90,90)
    plt.xlim(-180,180)
    plt.title(name + " " + desc)
    plt.figtext(0.5, 0.15, "Minimale Breite: " + str(min) + "° / maximale Breite: " + str(max) + "°", ha = "center", fontsize = 9, style = "italic")

    plt.savefig("Export/" + name + "_" + desc + '.png')

In [5]:
station.observation


C1            L1            L2  \
Epoch               SV                                              
2011-02-01 00:00:00 R13  2.063353e+07  1.101819e+08  8.569705e+07   
                    G14  2.239005e+07  1.176605e+08  9.168355e+07   
                    G22  2.512760e+07  1.320464e+08  1.028933e+08   
                    G15  2.463675e+07  1.294670e+08  1.008834e+08   
                    G32  2.543652e+07  1.336698e+08           NaN   
...                               ...           ...           ...   
2011-02-01 23:59:30 R12  2.262496e+07  1.208585e+08  9.400109e+07   
                    R24  2.288944e+07  1.224001e+08  9.520012e+07   
                    G15  2.479661e+07  1.303070e+08  1.015380e+08   
                    R05  2.229663e+07  1.191883e+08  9.270203e+07   
                    G12  2.036440e+07  1.070157e+08  8.338884e+07   

                                   P1            P2    S1    S2  \
Epoch               SV                                            
2011-02-01 00:00:00 R13  2.063353e+07  2.063353e+07  52.0  46.0   
                    G14  2.239005e+07  2.239006e+07  46.0  31.0   
                    G22  2.512760e+07  2.512760e+07  39.0  16.0   
                    G15  2.463675e+07  2.463675e+07  39.0  18.0   
                    G32           NaN           NaN  32.0   NaN   
...                               ...           ...   ...   ...   
2011-02-01 23:59:30 R12  2.262496e+07  2.262497e+07  44.0  40.0   
                    R24  2.288944e+07  2.288944e+07  43.0  40.0   
                    G15  2.479661e+07  2.479661e+07  38.0  19.0   
                    R05  2.229663e+07  2.229663e+07  47.0  42.0   
                    G12  2.036440e+07  2.036440e+07  53.0  45.0   

                                      epoch   SYSTEM  
Epoch               SV                                
2011-02-01 00:00:00 R13 2011-02-01 00:00:00  GLONASS  
                    G14 2011-02-01 00:00:00      GPS  
                    G22 2011-02-01 00:00:00      GPS  
                    G15 2011-02-01 00:00:00      GPS  
                    G32 2011-02-01 00:00:00      GPS  
...                                     ...      ...  
2011-02-01 23:59:30 R12 2011-02-01 23:59:30  GLONASS  
                    R24 2011-02-01 23:59:30  GLONASS  
                    G15 2011-02-01 23:59:30      GPS  
                    R05 2011-02-01 23:59:30  GLONASS  
                    G12 2011-02-01 23:59:30      GPS  

[52756 rows x 9 columns]